# Fed-BioMed Researcher base example

This example uses MNIST dataset. Please check `README.md` file in `notebooks` directory for the instructions to load MNIST dataset and configure nodes.


## Define an experiment model and parameters

Declare a torch training plan MyTrainingPlan class to send for training on the node

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms


# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    
    # Defines and return model 
    def init_model(self, model_args):
        return self.Net(model_args = model_args)
    
    # Defines and return optimizer
    def init_optimizer(self, optimizer_args):
        return torch.optim.Adam(self.model().parameters(), lr = optimizer_args["lr"])
    
    # Declares and return dependencies
    def init_dependencies(self):
        deps = ["from torchvision import datasets, transforms"]
        return deps
    
    class Net(nn.Module):
        def __init__(self, model_args):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 32, 3, 1)
            self.conv2 = nn.Conv2d(32, 64, 3, 1)
            self.dropout1 = nn.Dropout(0.25)
            self.dropout2 = nn.Dropout(0.5)
            self.fc1 = nn.Linear(9216, 128)
            self.fc2 = nn.Linear(128, 10)

        def forward(self, x):
            x = self.conv1(x)
            x = F.relu(x)
            x = self.conv2(x)
            x = F.relu(x)
            x = F.max_pool2d(x, 2)
            x = self.dropout1(x)
            x = torch.flatten(x, 1)
            x = self.fc1(x)
            x = F.relu(x)
            x = self.dropout2(x)
            x = self.fc2(x)


            output = F.log_softmax(x, dim=1)
            return output

    def training_data(self):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.model().forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss
    from sklearn.metrics import accuracy_score

    def evaluate(self, data, target):
        with torch.no_grad():
            output = self.model().forward(data)
            pred = output.argmax(dim=1)
            acc = accuracy_score(target.cpu().numpy(), pred.cpu().numpy())
            print(f"[Evaluate] Accuracy del batch de validación: {acc:.4f}")
            return 1 #{ 'accuracy': acc }
        


### Model arguments and training arguments

In [2]:
from fedbiomed.common.metrics import MetricTypes
model_args = {}

training_args = {
    'loader_args': { 'batch_size': 48, }, 
    'optimizer_args': {
        "lr" : 1e-3
    },
    'test_ratio' : 0.25,
    #'test_batch_size': 64,
    'test_metric': MetricTypes.F1_SCORE,
    'test_on_global_updates': True,
    'test_on_local_updates': True,
    #'test_metric_args': {'average': 'marco'},
    # 'use_gpu': True,  # automatically falls back to cpu on nodes that don't support gpu
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}



### Create and run the experiment

In [3]:

from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 tensorboard=True,
                 node_selection_strategy=None)



2025-07-31 00:57:35,940 fedbiomed INFO - Starting researcher service...

2025-07-31 00:57:35,945 fedbiomed INFO - Waiting 3s for nodes to connect...

2025-07-31 00:57:37,733 fedbiomed DEBUG - Node: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 polling for the tasks

2025-07-31 00:57:38,950 fedbiomed INFO - Updating training data. This action will update FederatedDataset, and the nodes that will participate to the experiment.

2025-07-31 00:57:38,998 fedbiomed DEBUG - Node: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 polling for the tasks

2025-07-31 00:57:39,002 fedbiomed INFO - Node selected for training -> NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57

<function extract_symbols at 0x7f0de0c7fb50>


2025-07-31 00:57:39,005 fedbiomed DEBUG - Model file has been saved: /home/gersa/fedbiomed-dcm/Modulo_DICOM/fbm-researcher/var/experiments/Experiment_0017/model_037467c4-fd13-4062-bf4a-88386e87d6bd.py

Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.


2025-07-31 00:57:39,016 fedbiomed INFO - Removing tensorboard logs from previous experiment

In [4]:
from fedbiomed.researcher.config import config
tensorboard_dir = config.vars['TENSORBOARD_RESULTS_DIR']
tensorboard_dir

'/home/gersa/fedbiomed-dcm/Modulo_DICOM/fbm-researcher/runs'

In [5]:
# Uncomment fr tensorboard 
# %load_ext tensorboard
# %tensorboard --logdir "$tensorboard_dir"

In [6]:

exp.run()



2025-07-31 00:57:39,042 fedbiomed INFO - Sampled nodes in round 0 ['NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57']

<function extract_symbols at 0x7f0de0c7fb50>


2025-07-31 00:57:39,048 fedbiomed INFO - Sending request 
					 To: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 
					 Request: : TRAIN
 -----------------------------------------------------------------

2025-07-31 00:57:39,088 fedbiomed DEBUG - Node: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 polling for the tasks

2025-07-31 00:57:44,944 fedbiomed INFO - VALIDATION ON GLOBAL UPDATES 
					 NODE_ID: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 
					 Round 1 | Iteration: 1/1 (100%) | Samples: 15000/15000
 					 F1_SCORE: 0.027287 
					 ---------

...

<function extract_symbols at 0x7f0de0c7fb50>


2025-07-31 00:58:07,131 fedbiomed INFO - Sending request 
					 To: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 
					 Request: :Perform final validation on aggregated parameters 
 -----------------------------------------------------------------

2025-07-31 00:58:07,193 fedbiomed DEBUG - Node: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 polling for the tasks

2025-07-31 00:58:11,310 fedbiomed INFO - VALIDATION ON GLOBAL UPDATES 
					 NODE_ID: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 
					 Round 3 | Iteration: 1/1 (100%) | Samples: 15000/15000
 					 F1_SCORE: 0.959226 
					 ---------

2

In [7]:

exp.run_once(increase=True)



2025-07-31 00:58:11,433 fedbiomed DEBUG - Auto increasing total rounds for experiment from 2 to 3

2025-07-31 00:58:11,437 fedbiomed INFO - Sampled nodes in round 2 ['NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57']

<function extract_symbols at 0x7f0de0c7fb50>


2025-07-31 00:58:11,506 fedbiomed INFO - Sending request 
					 To: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 
					 Request: : TRAIN
 -----------------------------------------------------------------

2025-07-31 00:58:11,553 fedbiomed DEBUG - Node: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 polling for the tasks

...

2025-07-31 00:58:20,196 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 
					 Round 3 Epoch: 1 | Iteration: 90/100 (90%) | Samples: 4320/4800
 					 Loss: 0.053044 
					 ---------

2025-07-31 00:58:20,568 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 
					 Round 3 Epoch: 1 | Iteration: 100/100 (100%) | Samples: 4800/4800
 					 Loss: 0.166804 
					 ---------

2025-07-31 00:58:24,627 fedbiomed INFO - VALIDATION ON LOCAL UPDATES 
					 NODE_ID: NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 
					 Round 3 | Iteration: 1/1 (100%) | Samples: 15000/15000
 					 F1_SCORE: 0.968726 
					 ---------

2025-07-31 00:58:24,946 fedbiomed INFO - Nodes that successfully reply in round 2 ['NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57']

1

Save trained model to file

In [8]:
try: 
    exp.training_plan().export_model('./trained_model')
except Exception as e:
    print(e)

#### Display results

In [9]:

print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1]
for r in round_data.values():
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
        rtraining = r['timing']['rtime_training'],
        ptraining = r['timing']['ptime_training'],
        rtotal = r['timing']['rtime_total']))
print('\n')




List the training rounds :  dict_keys([0, 1, 2])

List the nodes for the last training round and their timings : 
	- NODE_8761223e-feb5-4fb9-9f0b-9238a6db2a57 :    
		rtime_training=3.37 seconds    
		ptime_training=27.49 seconds    
		rtime_total=12.42 seconds




Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [10]:

print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())




List the training rounds :  dict_keys([0, 1, 2])

Access the federated params for the last training round :
	- parameter data:  dict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])
